In [1]:
import numpy as np
import os
import pickle
from time import time

from datetime import datetime
from PIL import Image
import tensorflow as tf
from tensorflow.python.framework import ops

import param

In [7]:
# dim = param.dim
dim = 50

In [8]:
learning_rate = param.learning_rate
# batch_size = param.batch_size
batch_size = 1
iter_time = param.iter_time

In [9]:
lambda_Q = param.lambda_Q
lambda_c = param.lambda_c
lambda_Z = param.lambda_Z
sigma = param.sigma
filter_num = param.filter_num
filter_size = param.filter_size

In [10]:
with open('Amazon/img_array/Amazon_img_436616.pkl', 'rb') as f:
    sample_img_1 = pickle.load(f)

In [11]:
with open('Amazon/img_array/Amazon_img_436615.pkl', 'rb') as f:
    sample_img_2 = pickle.load(f)

In [12]:
input1 = np.expand_dims(sample_img_1, 0)
input2 = np.expand_dims(sample_img_2, 0)

In [13]:
data = np.append(input1, input2, axis = 0)

In [14]:
data.shape

(2, 64, 64, 3)

In [15]:
shape = sample_img_1.shape
print(shape)

(64, 64, 3)


In [16]:
input_shape = [batch_size, shape[0], shape[1], shape[2]]
input_filter_shape = [filter_size[0], filter_size[1], 3, filter_num]
hidden_filter_shape = [filter_size[0], filter_size[1], filter_num, filter_num]
output_filter_shape = [filter_size[0], filter_size[1], filter_num, 3]

In [17]:
fc_dim = None

In [18]:
ops.reset_default_graph()

In [19]:
x_data = tf.placeholder(dtype=tf.float32, shape = input_shape, name = "Image_input")

In [20]:
def denoise(x_data):
    noise = tf.Variable(tf.random_normal(shape = input_shape, name = 'noise'))
    Z0 = tf.add(x_data, noise)
    return Z0

In [21]:
with tf.name_scope("Denoise") as scope:
    Z0 = denoise(x_data)

In [22]:
def e_layer_1(Z0):
    Q1 = tf.Variable(tf.random_normal(shape= input_filter_shape, stddev=1/lambda_Q), name = 'Q1')
    c1 = tf.Variable(tf.random_normal(shape= [batch_size, shape[0], shape[1], filter_num], stddev=1/ lambda_c), name = 'c1')
    conv_output1 = tf.nn.conv2d(Z0, filter=Q1, strides= [batch_size, 1, 1, 1], padding='SAME', name = 'conv_output1')
    output1 = tf.nn.sigmoid(tf.add(conv_output1, c1), name = 'output1')
    Z1 = tf.random_normal(shape = [batch_size, shape[0], shape[1], filter_num], mean = output1, stddev = 1/ lambda_Z, name = 'Z1')
    return Q1, c1, output1, Z1

In [23]:
with tf.name_scope("e_layer_1") as scope:
    Q1, c1, output1, Z1 = e_layer_1(Z0)

In [ ]:
def e_layer_2(Z1):
    Q2 = tf.Variable(tf.random_normal(shape= hidden_filter_shape, stddev=1/lambda_Q), name = 'Q2')
    c2 = tf.Variable(tf.random_normal(shape= [batch_size, shape[0], shape[1], filter_num], stddev=1/ lambda_c), name = 'c2')
    conv_output2 = tf.nn.conv2d(Z1, filter=Q2, strides= [batch_size, 1, 1, 1], padding='SAME', name = 'conv_output2')
    output2 = tf.nn.sigmoid(tf.add(conv_output2, c2), name = 'output2')
    Z2 = tf.random_normal(shape = [batch_size, shape[0], shape[1], filter_num], mean = output2, stddev = 1/ lambda_Z, name = 'Z2')
    return Q2, c2, output2, Z2

In [ ]:
with tf.name_scope("e_layer_2") as scope:
    Q2, c2, output2, Z2 = e_layer_2(Z1)

In [24]:
def fc_e_layer(Z2):
    Z2_flat = tf.reshape(Z2, shape=[-1])
    Z2_flat = tf.expand_dims(Z2_flat, 0)
    fc_dim = Z2_flat.get_shape().as_list()[1]
    Q3 = tf.Variable(tf.random_normal(shape = [fc_dim, dim], stddev=1 / lambda_Q), name = 'Q3')
    c3 = tf.Variable(tf.random_normal(shape = [dim], stddev = 1/ lambda_c), name = 'c3')
    output3 = tf.nn.sigmoid(tf.add(tf.matmul(Z2_flat, Q3), c3), name = 'output3')
    Z3_ = tf.random_normal(shape = [dim], mean = output3, stddev=1/ lambda_Z)
    return Q3, c3, output3, Z3_, fc_dim

In [25]:
with tf.name_scope("fc_e_layer") as scope:
    Q3, c3, output3, Z3_, fc_dim = fc_e_layer(Z1)

In [26]:
def fc_d_layer(Z3_):
    Q4 = tf.Variable(tf.random_normal(shape = [dim, fc_dim], stddev = 1/ lambda_Q), name = 'Q4')
    c4 = tf.Variable(tf.random_normal(shape = [fc_dim], stddev = 1/ lambda_c), name = 'c4')
    output4 = tf.nn.sigmoid(tf.add(tf.matmul(Z3_, Q4), c4), name = 'output4')
    Z4 = tf.random_normal(shape = [fc_dim], mean = output4, stddev = 1/ lambda_Z, name = 'Z4')
    return Q4, c4, output4, Z4

In [27]:
Q4, c4, output4, Z4 = fc_d_layer(Z3_)

In [28]:
Z4_mtx = tf.reshape(Z4, shape = [batch_size, shape[0], shape[1], filter_num])

In [ ]:
def d_layer_1(Z4):
    Q5 = tf.Variable(tf.random_normal(shape = hidden_filter_shape, stddev=1/ lambda_Q), name = 'Q5')
    c5 = tf.Variable(tf.random_normal(shape = [batch_size, shape[0], shape[1], filter_num], stddev= 1/ lambda_c), name = 'c5')
    conv_output5 = tf.nn.conv2d(Z4_mtx, filter = Q5, strides=[batch_size, 1, 1, 1], padding = 'SAME', name='conv_output5')
    output5 = tf.nn.sigmoid(tf.add(conv_output5, c5), name = 'output5')
    Z5 = tf.random_normal(shape = [batch_size, shape[0], shape[1], filter_num], stddev = 1/ lambda_Z, name = 'Z5')
    return Q5, c5, output5, Z5

In [ ]:
Q5, c5, output5, Z5 = d_layer_1(Z4_mtx)

In [29]:
def d_layer_2(Z5):
    Q6 = tf.Variable(tf.random_normal(shape = output_filter_shape, stddev=1/ lambda_Q), name = 'Q6')
    c6 = tf.Variable(tf.random_normal(shape = [batch_size, shape[0], shape[1], 3], stddev= 1/ lambda_c), name = 'c6')
    conv_output6 = tf.nn.conv2d(Z5, filter = Q6, strides=[batch_size, 1, 1, 1], padding = 'SAME', name='conv_output6')
    output6 = tf.nn.sigmoid(tf.add(conv_output6, c6), name = 'output6')
    Z6 = tf.random_normal(shape = [batch_size, shape[0], shape[1], 3], stddev = 1/ lambda_Z, name = 'Z6')
    return Q6, c6, output6, Z6

In [30]:
Q6, c6, output6, Z6 = d_layer_2(Z4_mtx)

In [31]:
log_likelihood = -tf.multiply(lambda_Z, tf.add_n([tf.reduce_mean(tf.square(tf.subtract(output1, Z1))),
#                                                  tf.reduce_mean(tf.square(tf.subtract(output2, Z2))), 
                                                 tf.reduce_mean(tf.square(tf.subtract(output3, Z3_))),
                                                 tf.reduce_mean(tf.square(tf.subtract(output4, Z4))),
#                                                  tf.reduce_mean(tf.square(tf.subtract(output5, Z5))),
                                                 tf.reduce_mean(tf.square(tf.subtract(output6, Z6)))]))/2 - tf.add_n([tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q1))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c1)))),
#                                                                                                                      tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q2))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c2)))),
                                                                                                                     tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q3))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c3)))),
                                                                                                                     tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q4))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c4)))),
#                                                                                                                      tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q5))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c5)))),
                                                                                                                     tf.add(tf.multiply(lambda_Q, tf.reduce_mean(tf.square(Q6))), tf.multiply(lambda_c, tf.reduce_mean(tf.square(c6))))])

loss = -log_likelihood

In [32]:
opt = tf.train.AdamOptimizer(learning_rate)
train = opt.minimize(loss)

In [33]:
init = tf.global_variables_initializer()

In [34]:
model_id = datetime.fromtimestamp(time()).strftime('%Y-%m-%d_%H:%M:%S')

In [35]:
save_dir = './SCAE/%s' %(model_id)
save_summary_path = os.path.join(save_dir, 'model_summary')
save_variable_path = os.path.join(save_dir, 'model_variables')

In [36]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    os.makedirs(save_summary_path)
    os.makedirs(save_variable_path)

In [37]:
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
sess = tf.Session(config = config)
sess.run(init)

/home/jihyung/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding
  from ipykernel import kernelapp as app


ResourceExhaustedError: OOM when allocating tensor with shape[50]
	 [[Node: fc_e_layer/random_normal_1/RandomStandardNormal = RandomStandardNormal[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](fc_e_layer/random_normal_1/shape)]]

Caused by op 'fc_e_layer/random_normal_1/RandomStandardNormal', defined at:
  File "/home/jihyung/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jihyung/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jihyung/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/jihyung/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jihyung/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/

In [ ]:
for i in range(iter_time):
    for (dirpath, dirnames, fnames) in os.walk('./Amazon/img_array/'):
        for fname in fnames:
            with open(os.path.join(dirpath, fname), 'rb') as f:
                X = pickle.load(f)
        X = np.expand_dims(data[i], 0)
        sess.run(train, feed_dict = {x_data:X})
        print(sess.run(loss, feed_dict={x_data:X}))
        break

In [ ]:
for v in tf.trainable_variables():
    print(v.name, end = ' ')
    print(v.get_shape())
    fname = v.name.replace('/', '-')
    fname = '{}.csv'.format(fname)
    fname = os.path.join(save_variable_path, fname)
    np.savetext(fname, v.eval(session=sess), delimiter=',')